<a href="https://colab.research.google.com/github/tom-pollak/crosscoders/blob/main/Caching_Gemma_2B_Activations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/jbloomAus/SAELens/pull/367
!git clone https://github.com/tom-pollak/SAELens.git -b dev/faster-sharded-dataset-cleanup
%cd SAELens
%pip install -qqq -e .
%cd /content/

fatal: destination path 'SAELens' already exists and is not an empty directory.
/content/SAELens
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sae-lens (pyproject.toml) ... done
/content


In [1]:
import math
import torch
from datasets import load_dataset

from sae_lens.cache_activations_runner import CacheActivationsRunner
from sae_lens.config import CacheActivationsRunnerConfig

In [ ]:
#@title Dataset
dataset_path = "ckkissane/pile-lmsys-mix-1m-tokenized-gemma-2"

ds = load_dataset(dataset_path, split="train")
context_size = ds['input_ids'][0].shape
ds_num_tokens = len(ds) * context_size
del ds

print(f"{ds_num_tokens=}, {context_size=}")

In [2]:
!rm -rf /content/cached-acts-gemma-2-2b

In [3]:
# DEBUG
# dataset_path = "ckkissane/pile-lmsys-mix-1m-tokenized-gemma-2"
# context_size = 1024
# ds_num_tokens = 926_000 * context_size

In [4]:
#@title Setup
n_devices = 1

model_name = "gemma-2-2b"
hook_layer = 14
hook_name = f"blocks.{hook_layer}.hook_resid_pre"
d_in = 2304

prepend_bos = False # In this case dataset already prepends BOS

cached_acts_path = f"/content/cached-acts-{model_name}"
hf_repo_id = f"tommyp111/{model_name}-pile-lymsys-mix-1m-14-resid-pre"

store_batch_size = 16 # model batch size (GPU VRAM)
batches_in_buffer = 128 # how large each buffer is (System RAM)

tokens_in_buffer = batches_in_buffer * store_batch_size * context_size

# Control how much of the dataset is used
num_buffers = math.ceil(ds_num_tokens / tokens_in_buffer) # whole dataset

num_tokens = tokens_in_buffer * num_buffers
assert num_tokens > ds_num_tokens

print(f"{tokens_in_buffer=} | {num_buffers=} | {num_tokens=}")


cfg = CacheActivationsRunnerConfig(
    new_cached_activations_path=cached_acts_path,
    model_name=model_name,
    hook_name=hook_name,
    dataset_path=dataset_path,
    training_tokens=num_tokens,
    shuffle=True,
    store_batch_size_prompts=store_batch_size,
    n_batches_in_buffer=batches_in_buffer,
    ###
    hook_layer=hook_layer,
    d_in=d_in,
    context_size=context_size,
    is_dataset_tokenized=True,
    prepend_bos=prepend_bos,
    normalize_activations="none",
    device="cuda",
    model_from_pretrained_kwargs=dict(n_devices=n_devices),
    seed=42,
    dtype="bfloat16",
    act_store_device="cpu",
    ###
    hf_repo_id=hf_repo_id
)

runner = CacheActivationsRunner(cfg)

tokens_in_buffer=1048576 | num_buffers=905 | num_tokens=948961280


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [5]:
#@title Get activations!
dataset = runner.run()

Started caching 948961280 activations


Caching activations:   0%|          | 0/905 [01:04<?, ?it/s]


KeyboardInterrupt: 